# ConveyorGuard - 04. DL Tuning (Optuna 하이퍼파라미터 최적화)

**Optuna 기반 하이퍼파라미터 최적화 → 95%+ 목표**

- **Accelerator: GPU T4 x2**
- 입력: `train_data.joblib`, `val_data.joblib`, `test_data.joblib`
- 출력: `tuned_model.pt`, `optuna_study.pkl`

---

### Pipeline
> `00_eda` → `01_preprocess` → `02/03`(병렬) → **`04_dl_tuning`** → `05_llm_comparison`

| 입력 데이터셋 | 출력 | Accelerator |
|--------------|------|-------------|
| `conveyorguard-preprocess` (01 출력) | `tuned_model.pt`, `optuna_study.pkl` | GPU T4 x2 |

### 모델 아키텍처
```
Sensor (30,8) → MLP → TemporalEncoder
                            ↓
                      CrossAttention → FiLM(External) → Classifier → 4-Class
                            ↑
Image (10,60,80) → CNN → TemporalEncoder
```

## 1. 환경 설정

In [1]:
# 패키지 설치
!pip install optuna plotly -q

import os
import sys
import numpy as np
import joblib
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import optuna
from optuna.trial import TrialState
import time
import warnings
import gc
warnings.filterwarnings('ignore')

# Plotly 설정
import plotly.io as pio
pio.renderers.default = 'iframe'

# 환경 정보
print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.version.cuda}")

# GPU 확인 (T4 x2 지원)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🖥️ Device: {device}")
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"   GPU {i}: {props.name} ({getattr(props, 'total_memory', None) or getattr(props, 'total_mem', 0) / 1024**3:.1f} GB)")
    print(f"   Total GPUs: {torch.cuda.device_count()}")

# ⚡ 속도 최적화
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('medium')
print(f"   cuDNN benchmark: enabled")
print(f"   float32 matmul precision: medium")

print(f"\n✅ 라이브러리 로드 완료")

# 전체 실행 시간 측정 시작
_notebook_start = time.time()

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch: 2.8.0+cu126
CUDA: 12.6

🖥️ Device: cuda
   GPU 0: Tesla T4 (15637086208.0 GB)
   GPU 1: Tesla T4 (15637086208.0 GB)
   Total GPUs: 2
   cuDNN benchmark: enabled
   float32 matmul precision: medium

✅ 라이브러리 로드 완료


In [2]:
# 경로 설정 (fallback 지원)
DATA_DIR = Path('/kaggle/input/conveyorguard-preprocess')
if not DATA_DIR.exists():
    DATA_DIR = Path('/kaggle/input/preprocess')
OUTPUT_DIR = Path('/kaggle/working')

print(f"📂 데이터 폴더: {DATA_DIR}")
print(f"📁 출력 폴더: {OUTPUT_DIR}")

📂 데이터 폴더: /kaggle/input/conveyorguard-preprocess
📁 출력 폴더: /kaggle/working


## 2. 데이터 로드

In [3]:
# joblib 로드
print("📂 데이터 로드 중...")

train_data = joblib.load(DATA_DIR / 'train_data.joblib')
val_data = joblib.load(DATA_DIR / 'val_data.joblib')
test_data = joblib.load(DATA_DIR / 'test_data.joblib')

print(f"\n📊 데이터 크기:")
print(f"   Train: {train_data['sensors'].shape[0]:,}개")
print(f"   Val: {val_data['sensors'].shape[0]:,}개")
print(f"   Test: {test_data['sensors'].shape[0]:,}개")

# 클래스 분포
print(f"\n📈 클래스 분포 (Train):")
unique, counts = np.unique(train_data['labels'], return_counts=True)
for u, c in zip(unique, counts):
    print(f"   Class {u}: {c:,} ({c/len(train_data['labels'])*100:.1f}%)")

📂 데이터 로드 중...

📊 데이터 크기:
   Train: 7,311개
   Val: 1,554개
   Test: 1,608개

📈 클래스 분포 (Train):
   Class 0: 3,586 (49.0%)
   Class 1: 1,682 (23.0%)
   Class 2: 1,642 (22.5%)
   Class 3: 401 (5.5%)


## 3. Dataset (최적화: 이미지 서브샘플링)

In [4]:
class ConveyorDatasetOptimized(Dataset):
    """
    최적화된 Dataset
    - 이미지: 30프레임 → 10프레임 서브샘플링 (3배 빨라짐)
    """
    def __init__(self, data: dict, image_frames: int = 10):
        self.sensors = torch.FloatTensor(data['sensors'])
        
        # ⚡ 이미지 서브샘플링 (30 → 10 프레임)
        images = data['images']
        indices = np.linspace(0, images.shape[1]-1, image_frames, dtype=int)
        self.images = torch.FloatTensor(images[:, indices, :, :])
        
        self.externals = torch.FloatTensor(data['external'])
        self.labels = torch.LongTensor(data['labels'])
        
        self.image_frames = image_frames
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            'sensors': self.sensors[idx],
            'images': self.images[idx],
            'externals': self.externals[idx],
            'label': self.labels[idx]
        }

# Dataset 생성
IMAGE_FRAMES = 10  # 30 → 10 (최적화)

train_dataset = ConveyorDatasetOptimized(train_data, image_frames=IMAGE_FRAMES)
val_dataset = ConveyorDatasetOptimized(val_data, image_frames=IMAGE_FRAMES)
test_dataset = ConveyorDatasetOptimized(test_data, image_frames=IMAGE_FRAMES)

print(f"✅ Dataset 생성 완료 (이미지 {IMAGE_FRAMES}프레임)")
print(f"   Train: {len(train_dataset):,}개")
print(f"   Val: {len(val_dataset):,}개")
print(f"   Test: {len(test_dataset):,}개")
print(f"   이미지 shape: {train_dataset.images.shape}")

✅ Dataset 생성 완료 (이미지 10프레임)
   Train: 7,311개
   Val: 1,554개
   Test: 1,608개
   이미지 shape: torch.Size([7311, 10, 60, 80])


## 4. 모델 정의 (이미지 프레임 수 조정)

In [5]:
class SensorEncoder(nn.Module):
    """센서 데이터 인코더 (MLP)"""
    def __init__(self, input_dim=8, embed_dim=128, dropout=0.1):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.LayerNorm(64),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(64, embed_dim),
            nn.LayerNorm(embed_dim),
            nn.GELU()
        )
    
    def forward(self, x):
        return self.mlp(x)


class ImageEncoder(nn.Module):
    """열화상 이미지 인코더 (CNN)"""
    def __init__(self, embed_dim=128, dropout=0.1):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.GELU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(dropout),
            
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.GELU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(dropout),
            
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.GELU(),
            nn.AdaptiveAvgPool2d(1)
        )
        self.proj = nn.Linear(128, embed_dim)
    
    def forward(self, x):
        B, T, H, W = x.shape
        x = x.view(B * T, 1, H, W)
        x = self.cnn(x)
        x = x.view(B * T, -1)
        x = self.proj(x)
        return x.view(B, T, -1)


class TemporalEncoder(nn.Module):
    """시계열 Transformer 인코더"""
    def __init__(self, embed_dim=128, num_heads=4, num_layers=2, dropout=0.1, seq_len=30):
        super().__init__()
        self.pos_embed = nn.Parameter(torch.randn(1, seq_len, embed_dim) * 0.02)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=embed_dim * 4,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
    
    def forward(self, x):
        x = x + self.pos_embed[:, :x.size(1), :]
        return self.transformer(x)


class CrossAttentionFusion(nn.Module):
    """Cross-Attention 기반 멀티모달 융합"""
    def __init__(self, embed_dim=128, num_heads=4, dropout=0.1):
        super().__init__()
        self.cross_attn = nn.MultiheadAttention(
            embed_dim=embed_dim,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True
        )
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 4),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(embed_dim * 4, embed_dim),
            nn.Dropout(dropout)
        )
    
    def forward(self, sensor_feat, image_feat):
        attn_out, _ = self.cross_attn(sensor_feat, image_feat, image_feat)
        x = self.norm1(sensor_feat + attn_out)
        x = self.norm2(x + self.ffn(x))
        return x


class ConveyorGuardModel(nn.Module):
    """전체 모델 (3-modal: 센서 + 열화상 + 외부환경)"""
    def __init__(self, embed_dim=128, num_heads=4, num_layers=2, dropout=0.1, 
                 num_classes=4, sensor_seq_len=30, image_seq_len=10):
        super().__init__()
        
        self.sensor_encoder = SensorEncoder(input_dim=8, embed_dim=embed_dim, dropout=dropout)
        self.image_encoder = ImageEncoder(embed_dim=embed_dim, dropout=dropout)
        self.external_encoder = nn.Sequential(
            nn.Linear(3, embed_dim),
            nn.LayerNorm(embed_dim),
            nn.GELU()
        )
        self.sensor_temporal = TemporalEncoder(embed_dim=embed_dim, num_heads=num_heads, 
                                                num_layers=num_layers, dropout=dropout, seq_len=sensor_seq_len)
        self.image_temporal = TemporalEncoder(embed_dim=embed_dim, num_heads=num_heads, 
                                               num_layers=num_layers, dropout=dropout, seq_len=image_seq_len)
        self.fusion = CrossAttentionFusion(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout)
        
        # External 조건화 (FiLM: Feature-wise Linear Modulation)
        self.film_gamma = nn.Linear(embed_dim, embed_dim)
        self.film_beta = nn.Linear(embed_dim, embed_dim)
        
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.LayerNorm(embed_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(embed_dim, num_classes)
        )
    
    def forward(self, sensors, images, externals=None):
        sensor_feat = self.sensor_encoder(sensors)
        image_feat = self.image_encoder(images)
        sensor_feat = self.sensor_temporal(sensor_feat)
        image_feat = self.image_temporal(image_feat)
        fused = self.fusion(sensor_feat, image_feat)
        pooled = fused.mean(dim=1)
        
        # FiLM 조건화: 외부환경으로 센서-이미지 융합 결과를 변조
        if externals is not None:
            ext_feat = self.external_encoder(externals)
            ext_pooled = ext_feat.mean(dim=1)
            gamma = self.film_gamma(ext_pooled)
            beta = self.film_beta(ext_pooled)
            pooled = gamma * pooled + beta
        
        return self.classifier(pooled)

print("✅ 모델 클래스 정의 완료 (FiLM: Feature-wise Linear Modulation)")

✅ 모델 클래스 정의 완료 (FiLM: Feature-wise Linear Modulation)


## 5. GPU 최적화 학습 함수

In [6]:
# 클래스 가중치 계산
train_labels = train_data['labels']
class_counts = np.bincount(train_labels)
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum() * len(class_counts)
class_weights_tensor = torch.FloatTensor(class_weights)
print(f"📊 클래스 가중치: {class_weights.round(2)}")

📊 클래스 가중치: [0.28 0.6  0.61 2.51]


In [7]:
def train_epoch_optimized(model, loader, criterion, optimizer, device, scaler=None):
    """GPU 최적화 학습 함수 (AMP + DataParallel)"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in loader:
        sensors = batch['sensors'].to(device, non_blocking=True)
        images = batch['images'].to(device, non_blocking=True)
        externals = batch['externals'].to(device, non_blocking=True)
        labels = batch['label'].to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        if scaler is not None:
            with autocast():
                logits = model(sensors, images, externals)
                loss = criterion(logits, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            logits = model(sensors, images, externals)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

        total_loss += loss.item() * labels.size(0)
        correct += (logits.argmax(1) == labels).sum().item()
        total += labels.size(0)

    return total_loss / total, correct / total * 100


def evaluate_optimized(model, loader, criterion, device):
    """GPU 평가 함수"""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in loader:
            sensors = batch['sensors'].to(device, non_blocking=True)
            images = batch['images'].to(device, non_blocking=True)
            externals = batch['externals'].to(device, non_blocking=True)
            labels = batch['label'].to(device, non_blocking=True)

            with autocast():
                logits = model(sensors, images, externals)
                loss = criterion(logits, labels)

            total_loss += loss.item() * labels.size(0)
            correct += (logits.argmax(1) == labels).sum().item()
            total += labels.size(0)

    return total_loss / total, correct / total * 100

print("✅ 학습/평가 함수 정의 완료 (AMP + DataParallel)")

✅ 학습/평가 함수 정의 완료 (AMP + DataParallel)


## 6. Optuna Objective 함수 (최적화)

In [8]:
# 고정 하이퍼파라미터
FIXED_BATCH_SIZE = 64  # T4 x2: GPU당 32
FIXED_EPOCHS = 10      # 빠른 탐색
FIXED_PATIENCE = 3     # Early stopping
NUM_WORKERS = 4
IMAGE_FRAMES = 10

def objective(trial):
    """Optuna objective 함수 (GPU T4 x2 + AMP)"""
    
    # 하이퍼파라미터 샘플링
    embed_dim = trial.suggest_categorical('embed_dim', [128, 256])
    num_heads = trial.suggest_categorical('num_heads', [4, 8])
    num_layers = trial.suggest_int('num_layers', 1, 2)
    dropout = trial.suggest_float('dropout', 0.1, 0.3, step=0.1)
    lr = trial.suggest_float('lr', 1e-4, 1e-3, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)
    
    # DataLoader
    train_loader = DataLoader(train_dataset, batch_size=FIXED_BATCH_SIZE,
                              shuffle=True, num_workers=NUM_WORKERS,
                              pin_memory=True, persistent_workers=True,
                              drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=FIXED_BATCH_SIZE,
                            shuffle=False, num_workers=NUM_WORKERS,
                            pin_memory=True, persistent_workers=True)
    
    # 모델 생성
    model = ConveyorGuardModel(
        embed_dim=embed_dim,
        num_heads=num_heads,
        num_layers=num_layers,
        dropout=dropout,
        num_classes=4,
        image_seq_len=IMAGE_FRAMES
    )
    
    # ⚡ DataParallel (T4 x2)
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model = model.to(device)
    
    # AMP Scaler
    scaler = GradScaler()
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor.to(device))
    
    best_acc = 0
    no_improve = 0
    
    for epoch in range(FIXED_EPOCHS):
        train_loss, train_acc = train_epoch_optimized(model, train_loader, criterion, optimizer, device, scaler)
        val_loss, val_acc = evaluate_optimized(model, val_loader, criterion, device)
        
        trial.report(val_acc, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()
        
        if val_acc > best_acc:
            best_acc = val_acc
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= FIXED_PATIENCE:
                break
    
    # 메모리 정리
    del model, optimizer, criterion, scaler
    gc.collect()
    torch.cuda.empty_cache()
    
    return best_acc

print(f"✅ Objective 함수 정의 완료")
print(f"   Batch size: {FIXED_BATCH_SIZE} (GPU당 {FIXED_BATCH_SIZE // max(torch.cuda.device_count(), 1)})")
print(f"   DataParallel: {torch.cuda.device_count() > 1}")

✅ Objective 함수 정의 완료
   Batch size: 64 (GPU당 32)
   DataParallel: True


## 7. Optuna 최적화 실행

In [9]:
# ⚡ 최적화된 설정
N_TRIALS = 15  # 30 → 15 (시간 단축)
TIMEOUT = 1800  # 30분 타임아웃

# Optuna Study 생성
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=3)
)

print("🔍 하이퍼파라미터 탐색 시작 (GPU T4 x2 최적화)...")
print(f"   목표: Val Accuracy 최대화")
print(f"   Trial 수: {N_TRIALS}")
print(f"   타임아웃: {TIMEOUT//60}분")
print("="*60)

start_time = time.time()

# 최적화 실행
study.optimize(
    objective, 
    n_trials=N_TRIALS,
    timeout=TIMEOUT,
    show_progress_bar=True,
    gc_after_trial=True
)

total_time = (time.time() - start_time) / 60
print("="*60)
print(f"\n⏱️ 총 탐색 시간: {total_time:.1f}분")

[I 2026-02-01 07:32:12,093] A new study created in memory with name: no-name-e9727a0a-1395-4375-a29b-0ed7e8da3c56


🔍 하이퍼파라미터 탐색 시작 (GPU T4 x2 최적화)...
   목표: Val Accuracy 최대화
   Trial 수: 15
   타임아웃: 30분


  0%|          | 0/15 [00:00<?, ?it/s]

[I 2026-02-01 07:36:01,187] Trial 0 finished with value: 88.73873873873875 and parameters: {'embed_dim': 256, 'num_heads': 4, 'num_layers': 1, 'dropout': 0.1, 'lr': 0.00011430983876313228, 'weight_decay': 0.0005399484409787432}. Best is trial 0 with value: 88.73873873873875.
[I 2026-02-01 07:41:00,093] Trial 1 finished with value: 89.83268983268984 and parameters: {'embed_dim': 256, 'num_heads': 8, 'num_layers': 2, 'dropout': 0.1, 'lr': 0.0001519934830130982, 'weight_decay': 2.3270677083837777e-05}. Best is trial 1 with value: 89.83268983268984.
[I 2026-02-01 07:46:33,955] Trial 2 finished with value: 90.47619047619048 and parameters: {'embed_dim': 256, 'num_heads': 4, 'num_layers': 2, 'dropout': 0.1, 'lr': 0.00019594972058679176, 'weight_decay': 5.4041038546473305e-05}. Best is trial 2 with value: 90.47619047619048.
[I 2026-02-01 07:50:59,158] Trial 3 finished with value: 90.09009009009009 and parameters: {'embed_dim': 256, 'num_heads': 8, 'num_layers': 2, 'dropout': 0.1, 'lr': 0.0004

In [10]:
# 최적 결과 출력
print("\n" + "="*60)
print("🏆 최적 하이퍼파라미터")
print("="*60)

best_trial = study.best_trial
print(f"\n📊 Best Val Accuracy: {best_trial.value:.2f}%")
print(f"\n📋 Best Parameters:")
for key, value in best_trial.params.items():
    print(f"   {key}: {value}")

# 상위 5개 Trial
print(f"\n📈 Top 5 Trials:")
trials_df = study.trials_dataframe()
trials_df = trials_df.sort_values('value', ascending=False)
print(trials_df[['number', 'value', 'params_embed_dim', 'params_lr', 'params_dropout']].head())


🏆 최적 하이퍼파라미터

📊 Best Val Accuracy: 90.48%

📋 Best Parameters:
   embed_dim: 256
   num_heads: 4
   num_layers: 2
   dropout: 0.1
   lr: 0.00019594972058679176
   weight_decay: 5.4041038546473305e-05

📈 Top 5 Trials:
   number      value  params_embed_dim  params_lr  params_dropout
2       2  90.476190               256   0.000196             0.1
3       3  90.090090               256   0.000405             0.1
1       1  89.832690               256   0.000152             0.1
0       0  88.738739               256   0.000114             0.1
5       5  86.486486               256   0.000205             0.2


## 8. 최적 모델 재학습 (더 많은 Epoch)

In [11]:
# 최적 파라미터로 모델 재학습
print("\n🚀 최적 파라미터로 최종 모델 학습...")

best_params = best_trial.params

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=FIXED_BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=FIXED_BATCH_SIZE, shuffle=False,
                        num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)
test_loader = DataLoader(test_dataset, batch_size=FIXED_BATCH_SIZE, shuffle=False,
                         num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)

# 모델 생성
model = ConveyorGuardModel(
    embed_dim=best_params['embed_dim'],
    num_heads=best_params['num_heads'],
    num_layers=best_params['num_layers'],
    dropout=best_params['dropout'],
    num_classes=4,
    image_seq_len=IMAGE_FRAMES
)

# ⚡ DataParallel (T4 x2)
if torch.cuda.device_count() > 1:
    print(f"⚡ Using {torch.cuda.device_count()} GPUs with DataParallel")
    model = nn.DataParallel(model)
model = model.to(device)

# AMP Scaler
scaler = GradScaler()

optimizer = torch.optim.AdamW(model.parameters(),
                              lr=best_params['lr'],
                              weight_decay=best_params['weight_decay'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor.to(device))

# 학습
N_EPOCHS = 30
PATIENCE = 7
best_acc = 0
no_improve = 0
best_model_state = None

print(f"\n학습 시작 (Epochs: {N_EPOCHS}, Patience: {PATIENCE})")
print(f"   Batch: {FIXED_BATCH_SIZE} (GPU당 {FIXED_BATCH_SIZE // max(torch.cuda.device_count(), 1)})")
print(f"   DataParallel: {isinstance(model, nn.DataParallel)}")
print("="*60)

start_time = time.time()

for epoch in range(1, N_EPOCHS + 1):
    train_loss, train_acc = train_epoch_optimized(model, train_loader, criterion, optimizer, device, scaler)
    val_loss, val_acc = evaluate_optimized(model, val_loader, criterion, device)
    scheduler.step()
    
    print(f"Epoch {epoch:2d}/{N_EPOCHS} │ Train: {train_loss:.4f}, {train_acc:.1f}% │ Val: {val_loss:.4f}, {val_acc:.1f}%", end='')
    
    if val_acc > best_acc:
        best_acc = val_acc
        no_improve = 0
        # DataParallel인 경우 model.module.state_dict() 사용
        model_to_save = model.module if isinstance(model, nn.DataParallel) else model
        best_model_state = model_to_save.state_dict().copy()
        print(f" ✅ Best!")
    else:
        no_improve += 1
        print(f" (no improve: {no_improve})")
    
    if no_improve >= PATIENCE:
        print(f"\n⚠️ Early stopping at epoch {epoch}")
        break

total_time = (time.time() - start_time) / 60
print("="*60)
print(f"\n⏱️ 학습 시간: {total_time:.1f}분")
print(f"🏆 Best Val Accuracy: {best_acc:.2f}%")


🚀 최적 파라미터로 최종 모델 학습...
⚡ Using 2 GPUs with DataParallel

학습 시작 (Epochs: 30, Patience: 7)
   Batch: 64 (GPU당 32)
   DataParallel: True
Epoch  1/30 │ Train: 0.5511, 77.0% │ Val: 0.3728, 81.0% ✅ Best!
Epoch  2/30 │ Train: 0.3568, 84.5% │ Val: 0.2961, 84.3% ✅ Best!
Epoch  3/30 │ Train: 0.3142, 86.1% │ Val: 0.3123, 85.8% ✅ Best!
Epoch  4/30 │ Train: 0.3057, 86.7% │ Val: 0.2670, 86.2% ✅ Best!
Epoch  5/30 │ Train: 0.2952, 87.0% │ Val: 0.2895, 86.0% (no improve: 1)
Epoch  6/30 │ Train: 0.2771, 87.6% │ Val: 0.2597, 89.1% ✅ Best!
Epoch  7/30 │ Train: 0.2613, 88.4% │ Val: 0.3048, 88.0% (no improve: 1)
Epoch  8/30 │ Train: 0.2499, 88.9% │ Val: 0.2210, 89.3% ✅ Best!
Epoch  9/30 │ Train: 0.2470, 88.8% │ Val: 0.2358, 87.3% (no improve: 1)
Epoch 10/30 │ Train: 0.2269, 89.6% │ Val: 0.2017, 90.0% ✅ Best!
Epoch 11/30 │ Train: 0.2302, 89.6% │ Val: 0.1920, 90.3% ✅ Best!
Epoch 12/30 │ Train: 0.2277, 89.6% │ Val: 0.3115, 84.9% (no improve: 1)
Epoch 13/30 │ Train: 0.2102, 90.5% │ Val: 0.2465, 86.5% (no impro

## 8-1. Ablation Study (멀티모달 효과 검증)

각 모달리티의 기여도를 검증하기 위해 4가지 구성을 비교합니다.

| 구성 | 센서 | 이미지 | 외부환경(FiLM) |
|------|------|--------|---------------|
| Sensor Only | ✅ | - | - |
| Image Only | - | ✅ | - |
| Sensor + Image | ✅ | ✅ | - |
| Full (+ FiLM) | ✅ | ✅ | ✅ |

In [12]:
# Ablation 모델 정의

class SensorOnlyModel(nn.Module):
    """센서만 사용 (이미지 X, 외부환경 X)"""
    def __init__(self, embed_dim=128, num_heads=4, num_layers=2, dropout=0.1, num_classes=4):
        super().__init__()
        self.sensor_encoder = SensorEncoder(input_dim=8, embed_dim=embed_dim, dropout=dropout)
        self.temporal = TemporalEncoder(embed_dim=embed_dim, num_heads=num_heads,
                                        num_layers=num_layers, dropout=dropout, seq_len=30)
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, embed_dim), nn.LayerNorm(embed_dim),
            nn.GELU(), nn.Dropout(dropout), nn.Linear(embed_dim, num_classes)
        )
    
    def forward(self, sensors, images=None, externals=None):
        x = self.sensor_encoder(sensors)
        x = self.temporal(x)
        return self.classifier(x.mean(dim=1))


class ImageOnlyModel(nn.Module):
    """이미지만 사용 (센서 X, 외부환경 X)"""
    def __init__(self, embed_dim=128, num_heads=4, num_layers=2, dropout=0.1, 
                 num_classes=4, image_seq_len=10):
        super().__init__()
        self.image_encoder = ImageEncoder(embed_dim=embed_dim, dropout=dropout)
        self.temporal = TemporalEncoder(embed_dim=embed_dim, num_heads=num_heads,
                                        num_layers=num_layers, dropout=dropout, seq_len=image_seq_len)
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, embed_dim), nn.LayerNorm(embed_dim),
            nn.GELU(), nn.Dropout(dropout), nn.Linear(embed_dim, num_classes)
        )
    
    def forward(self, sensors=None, images=None, externals=None):
        x = self.image_encoder(images)
        x = self.temporal(x)
        return self.classifier(x.mean(dim=1))


class SensorImageModel(nn.Module):
    """센서+이미지 (FiLM 없음)"""
    def __init__(self, embed_dim=128, num_heads=4, num_layers=2, dropout=0.1, 
                 num_classes=4, image_seq_len=10):
        super().__init__()
        self.sensor_encoder = SensorEncoder(input_dim=8, embed_dim=embed_dim, dropout=dropout)
        self.image_encoder = ImageEncoder(embed_dim=embed_dim, dropout=dropout)
        self.sensor_temporal = TemporalEncoder(embed_dim=embed_dim, num_heads=num_heads,
                                                num_layers=num_layers, dropout=dropout, seq_len=30)
        self.image_temporal = TemporalEncoder(embed_dim=embed_dim, num_heads=num_heads,
                                               num_layers=num_layers, dropout=dropout, seq_len=image_seq_len)
        self.fusion = CrossAttentionFusion(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout)
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, embed_dim), nn.LayerNorm(embed_dim),
            nn.GELU(), nn.Dropout(dropout), nn.Linear(embed_dim, num_classes)
        )
    
    def forward(self, sensors, images, externals=None):
        sensor_feat = self.sensor_encoder(sensors)
        image_feat = self.image_encoder(images)
        sensor_feat = self.sensor_temporal(sensor_feat)
        image_feat = self.image_temporal(image_feat)
        fused = self.fusion(sensor_feat, image_feat)
        return self.classifier(fused.mean(dim=1))

print("✅ Ablation 모델 정의 완료 (SensorOnly, ImageOnly, SensorImage)")

✅ Ablation 모델 정의 완료 (SensorOnly, ImageOnly, SensorImage)


In [13]:
# 본 모델 GPU 메모리 해제 (Ablation용 공간 확보)
del model, optimizer, scheduler, criterion, scaler
gc.collect()
torch.cuda.empty_cache()
print("✅ 본 모델 GPU 메모리 해제 완료")

# Ablation 학습 루프
print("\n🔬 Ablation Study 시작...")
print("="*60)

ABLATION_EPOCHS = 10
ABLATION_PATIENCE = 3

ablation_configs = [
    ('Sensor Only', SensorOnlyModel, {'embed_dim': best_params['embed_dim'],
        'num_heads': best_params['num_heads'], 'num_layers': best_params['num_layers'],
        'dropout': best_params['dropout']}),
    ('Image Only', ImageOnlyModel, {'embed_dim': best_params['embed_dim'],
        'num_heads': best_params['num_heads'], 'num_layers': best_params['num_layers'],
        'dropout': best_params['dropout'], 'image_seq_len': IMAGE_FRAMES}),
    ('Sensor + Image', SensorImageModel, {'embed_dim': best_params['embed_dim'],
        'num_heads': best_params['num_heads'], 'num_layers': best_params['num_layers'],
        'dropout': best_params['dropout'], 'image_seq_len': IMAGE_FRAMES}),
]

ablation_results = {}

for name, ModelClass, kwargs in ablation_configs:
    print(f"\n▶ {name}")
    
    abl_model = ModelClass(**kwargs).to(device)
    abl_scaler = GradScaler()
    abl_optimizer = torch.optim.AdamW(abl_model.parameters(), lr=best_params['lr'],
                                       weight_decay=best_params['weight_decay'])
    abl_criterion = nn.CrossEntropyLoss(weight=class_weights_tensor.to(device))
    
    abl_best_acc = 0
    abl_no_improve = 0
    
    for epoch in range(1, ABLATION_EPOCHS + 1):
        train_loss, train_acc = train_epoch_optimized(
            abl_model, train_loader, abl_criterion, abl_optimizer, device, abl_scaler)
        val_loss, val_acc = evaluate_optimized(
            abl_model, val_loader, abl_criterion, device)
        
        if val_acc > abl_best_acc:
            abl_best_acc = val_acc
            abl_no_improve = 0
        else:
            abl_no_improve += 1
            if abl_no_improve >= ABLATION_PATIENCE:
                break
    
    ablation_results[name] = abl_best_acc
    print(f"   Best Val Acc: {abl_best_acc:.2f}%")
    
    del abl_model, abl_scaler, abl_optimizer, abl_criterion
    gc.collect()
    torch.cuda.empty_cache()

# Full 모델 결과 추가
ablation_results['Sensor + Image + FiLM (Full)'] = best_acc

print("\n" + "="*60)
print("📊 Ablation Study 결과:")
print("="*60)
for name, acc in ablation_results.items():
    bar = "█" * int(acc / 2)
    print(f"   {name:30s} → {acc:.2f}%  {bar}")
print("="*60)

✅ 본 모델 GPU 메모리 해제 완료

🔬 Ablation Study 시작...

▶ Sensor Only
   Best Val Acc: 89.12%

▶ Image Only
   Best Val Acc: 69.56%

▶ Sensor + Image
   Best Val Acc: 89.64%

📊 Ablation Study 결과:
   Sensor Only                    → 89.12%  ████████████████████████████████████████████
   Image Only                     → 69.56%  ██████████████████████████████████
   Sensor + Image                 → 89.64%  ████████████████████████████████████████████
   Sensor + Image + FiLM (Full)   → 90.35%  █████████████████████████████████████████████


## 9. Test 평가

In [14]:
# Best 모델 재생성 + 로드 (Ablation에서 메모리 해제했으므로 새로 생성)
from sklearn.metrics import classification_report, confusion_matrix

model = ConveyorGuardModel(
    embed_dim=best_params['embed_dim'],
    num_heads=best_params['num_heads'],
    num_layers=best_params['num_layers'],
    dropout=best_params['dropout'],
    num_classes=4,
    image_seq_len=IMAGE_FRAMES
).to(device)
model.load_state_dict({k: v.to(device) for k, v in best_model_state.items()})

criterion = nn.CrossEntropyLoss(weight=class_weights_tensor.to(device))

# Test 평가
test_loss, test_acc = evaluate_optimized(model, test_loader, criterion, device)

# Classification Report
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        sensors = batch['sensors'].to(device, non_blocking=True)
        images = batch['images'].to(device, non_blocking=True)
        externals = batch['externals'].to(device, non_blocking=True)
        labels = batch['label'].to(device, non_blocking=True)
        with autocast():
            logits = model(sensors, images, externals)
        all_preds.extend(logits.argmax(1).cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

label_names = ['정상', '경미', '중간', '심각']
print(f"\n🎯 Test Accuracy: {test_acc:.2f}%\n")
print(classification_report(all_labels, all_preds, target_names=label_names))


🎯 Test Accuracy: 87.75%

              precision    recall  f1-score   support

          정상       0.97      0.85      0.90       788
          경미       0.70      0.91      0.79       371
          중간       0.95      0.90      0.92       361
          심각       0.87      0.97      0.91        88

    accuracy                           0.88      1608
   macro avg       0.87      0.90      0.88      1608
weighted avg       0.90      0.88      0.88      1608



In [15]:
# Confusion Matrix
import plotly.figure_factory as ff

cm = confusion_matrix(all_labels, all_preds)
cm_text = [[str(y) for y in x] for x in cm]

fig = ff.create_annotated_heatmap(
    cm,
    x=label_names,
    y=label_names,
    annotation_text=cm_text,
    colorscale='Blues',
    showscale=True
)

fig.update_layout(
    title=f'Confusion Matrix (Test Acc: {test_acc:.2f}%)',
    xaxis_title='Predicted',
    yaxis_title='Actual',
    width=600,
    height=500
)

fig.show(renderer='iframe')
fig.write_html(OUTPUT_DIR / 'tuned_confusion_matrix.html')
print(f"\n📊 tuned_confusion_matrix.html 저장 완료")


📊 tuned_confusion_matrix.html 저장 완료


## 10. 모델 저장

In [16]:
# 모델 저장
best_model_path = OUTPUT_DIR / 'tuned_model.pt'

torch.save({
    'model_state_dict': best_model_state,
    'best_val_acc': best_acc,
    'test_acc': test_acc,
    'best_params': best_params,
    'ablation_results': ablation_results,
    'model_config': {
        'embed_dim': best_params['embed_dim'],
        'num_heads': best_params['num_heads'],
        'num_layers': best_params['num_layers'],
        'dropout': best_params['dropout'],
        'num_classes': 4,
        'sensor_seq_len': 30,
        'image_seq_len': IMAGE_FRAMES
    }
}, best_model_path)

# Optuna Study 저장
import pickle
with open(OUTPUT_DIR / 'optuna_study.pkl', 'wb') as f:
    pickle.dump(study, f)

print(f"✅ 모델 저장 완료: {best_model_path}")
print(f"   Best Val Acc: {best_acc:.2f}%")
print(f"   Test Acc: {test_acc:.2f}%")
print(f"   Ablation 결과 포함: {list(ablation_results.keys())}")

✅ 모델 저장 완료: /kaggle/working/tuned_model.pt
   Best Val Acc: 90.35%
   Test Acc: 87.75%
   Ablation 결과 포함: ['Sensor Only', 'Image Only', 'Sensor + Image', 'Sensor + Image + FiLM (Full)']


## Kaggle 데이터셋 업로드 (API)

In [17]:
import os, json

os.environ['KAGGLE_USERNAME'] = 'kukass'
os.environ['KAGGLE_KEY'] = 'KGAT_c973fff8eb3e1ccb19f3e9d683eb17dc'

UPLOAD_DIR = '/kaggle/working/dataset_upload'
os.makedirs(UPLOAD_DIR, exist_ok=True)

output_files = ['tuned_model.pt', 'optuna_study.pkl', 'tuned_confusion_matrix.html']
for f in output_files:
    src = f'/kaggle/working/{f}'
    dst = f'{UPLOAD_DIR}/{f}'
    if os.path.exists(src) and not os.path.exists(dst):
        os.symlink(src, dst)

meta = {
    "title": "conveyorguard-tuned",
    "id": "kukass/conveyorguard-tuned",
    "licenses": [{"name": "CC0-1.0"}]
}
with open(f'{UPLOAD_DIR}/dataset-metadata.json', 'w') as f:
    json.dump(meta, f)

!kaggle datasets create -p {UPLOAD_DIR} --dir-mode zip

print("\n✅ conveyorguard-tuned 데이터셋 업로드 완료!")
print("   → 06_ensemble, 07_final_comparison에서 사용 가능")

Starting upload for file optuna_study.pkl
100%|██████████████████████████████████████| 15.6k/15.6k [00:00<00:00, 39.1kB/s]
Upload successful: optuna_study.pkl (16KB)
Starting upload for file tuned_model.pt
100%|██████████████████████████████████████| 16.5M/16.5M [00:00<00:00, 29.9MB/s]
Upload successful: tuned_model.pt (16MB)
Starting upload for file tuned_confusion_matrix.html
100%|██████████████████████████████████████| 4.36M/4.36M [00:00<00:00, 9.49MB/s]
Upload successful: tuned_confusion_matrix.html (4MB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/kukass/conveyorguard-tuned

✅ conveyorguard-tuned 데이터셋 업로드 완료!
   → 06_ensemble, 07_final_comparison에서 사용 가능


---

## 📋 결과 요약

| 항목 | Baseline (02) | Tuned (04) |
|------|---------------|------------|
| Accelerator | GPU T4 x2 | GPU T4 x2 |
| 최적화 | 수동 | Optuna |
| DataParallel | ✅ | ✅ |
| AMP | ✅ | ✅ |
| FiLM | ✅ | ✅ |

### 🔬 Ablation Study
```
Sensor Only           → 단일 모달리티 한계 확인
Image Only            → 열화상만으로는 불충분
Sensor + Image        → 멀티모달 융합 효과
Sensor + Image + FiLM → 외부환경 조건화로 추가 성능 향상
```

### ⚡ GPU T4 x2 최적화
```
- DataParallel: 2x GPU 병렬 처리
- AMP (Mixed Precision): 메모리 절감 + 속도 향상
- cuDNN benchmark: 최적 커널 자동 선택
- 이미지 서브샘플링: 30 → 10 프레임
- Optuna Pruning: 비효율 Trial 조기 종료
```

### 다음 단계
- `05_llm_comparison.ipynb`: LLM 진단 비교 + LangGraph 멀티 에이전트